<a href="https://colab.research.google.com/github/hahw94/DNN-ResNet-etc/blob/master/1_%EA%B8%B0%EB%B3%B8%EC%A0%81%EC%9D%B8_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%8B%A4%EB%93%AC%EB%8A%94_%EB%B0%A9%EB%B2%95(GOCI_PM10%2C25_%EA%B4%80%EC%B8%A1%EC%86%8C%EB%B3%84_%EB%8D%B0%EC%9D%B4%ED%84%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# (코랩) 기본적인 데이터 다듬는 방법(GOCI PM10,25 / 관측소별 데이터)

In [1]:
'''
;Column 0 is site number
;Column 1 is year
;Column 2 is month
;Column 3 is day        
;Column 4 is hour       
;Column 5 is AOD         
;Column 6 is FMF        
;Column 7 is SSA        
;Column 8 is AE         
;Column 9 is NDVI        
;Column10 is DAI         
;Column11 is SO2        
;Column12 is CO         
;Column13 is O3          
;Column14 is NO2        
;Column15 is PM10       
;Column16 is PM25         
;Column17 is temp     
;Column18 is wd      
;Column19 is ws     
;Column20 is rh     
;Column21 is PBLH    
;Column22 is tcc    
'''

'\n;Column 0 is site number\n;Column 1 is year\n;Column 2 is month\n;Column 3 is day        \n;Column 4 is hour       \n;Column 5 is AOD         \n;Column 6 is FMF        \n;Column 7 is SSA        \n;Column 8 is AE         \n;Column 9 is NDVI        \n;Column10 is DAI         \n;Column11 is SO2        \n;Column12 is CO         \n;Column13 is O3          \n;Column14 is NO2        \n;Column15 is PM10       \n;Column16 is PM25         \n;Column17 is temp     \n;Column18 is wd      \n;Column19 is ws     \n;Column20 is rh     \n;Column21 is PBLH    \n;Column22 is tcc    \n'

# 1. 구글 드라이브 저장소 마운트 및 사용 할 라이브러리 불러오기
- 코랩에서 개인 데이터를 사용하기 위해서는 구글 드라이브에 본인이 사용할 데이터를 업로드 해야한다
- 업로드 한 데이터를 불러오기 위해서는 구글 드라이브 저장소를 마운트해서 사용할 수 있게 해주어야한다.
- 본인이 사용할 라이브러리는 코드를 짜기전 미리 전부 불러와야 한다.

In [33]:
from google.colab import drive 
drive.mount('/gdrive')
# 구글 드라이브 저장소 마운트 하기

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [34]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import time
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# 2. 데이터 불러오기
- 먼저 구글드라이브에 사용할 데이터를 업로드 하고 해당 디렉토리로 접속한 후 불러올 데이터들을 가져온다.

In [35]:
cd '/gdrive/My Drive/Colab Notebooks/Model/Dataset/PM10'

/gdrive/My Drive/Colab Notebooks/Model/Dataset/PM10


In [36]:
os.listdir()[:5]
# 해당 디렉토리의 처음부터 다섯번째까지의 파일을 출력해준다.

['111121_collection_pm10_2012to2019.txt',
 '111123_collection_pm10_2012to2019.txt',
 '111131_collection_pm10_2012to2019.txt',
 '111141_collection_pm10_2012to2019.txt',
 '111142_collection_pm10_2012to2019.txt']

In [37]:
df = pd.read_csv(os.listdir()[0],sep='    ')
# sep을 기준으로 데이터 프레임을 만드는데 해당 txt파일의 데이터가 4칸의 공백으로 띄어져있어 이렇게 만들었습니다.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [38]:
df.columns = ['site number','year','month','day','hour','AOD','FMF','SSA','AE','NDVI','DAI','SO2','CO','O3','NO2','PM10','PM25','temp','wd','ws','rh','PBLH','tcc']
# 다음과 같이 컬럼들에 이름을 붙여줍니다.(데이터에 해당하는 정보 입력)

In [39]:
df.head()
# 컬럼별 이름을 붙인 기본적인 데이터의 모습

,site number,year,month,day,hour,AOD,FMF,SSA,AE,NDVI,DAI,SO2,CO,O3,NO2,PM10,PM25,temp,wd,ws,rh,PBLH,tcc
0,111121.0,2012.0,1.0,1.0,1.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.011,0.6,0.011,0.023,83.0,-999.0,-1.2,290.0,3.1,65.0,994.30627,0.77436
1,111121.0,2012.0,1.0,1.0,2.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.012,0.6,0.015,0.020,75.0,-999.0,-1.1,290.0,4.2,60.0,1229.21326,0.86685
2,111121.0,2012.0,1.0,1.0,3.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.012,0.7,0.015,0.022,86.0,-999.0,-0.9,270.0,5.6,61.0,1178.61377,0.95999
3,111121.0,2012.0,1.0,1.0,4.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.012,0.6,0.019,0.019,91.0,-999.0,-1.1,290.0,5.1,49.0,1161.86670,0.80128
4,111121.0,2012.0,1.0,1.0,5.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.012,0.4,0.020,0.017,86.0,-999.0,-2.0,290.0,5.8,52.0,1190.97266,0.56124


# 3. 다양한 방법으로 필요한 데이터 뽑기
- 1) 필요없는 데이터 제거하기
- 2) 정규화 하기
- 3) Interpolate 하기

## 1) 필요없는 데이터 제거하기

In [40]:
df.columns
# df의 컬럼들을 모두 불러온다.

Index(['site number', 'year', 'month', 'day', 'hour', 'AOD', 'FMF', 'SSA',
       'AE', 'NDVI', 'DAI', 'SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25', 'temp',
       'wd', 'ws', 'rh', 'PBLH', 'tcc'],
      dtype='object')

In [41]:
# 위의 데이터에서 필요없는 데이터들을 고른다 (예 : 'site number','year','PM25')
# difference 함수를 사용해서 다음과 같이 필요없는 데이터들의 컬럼 제목을 써준다.
# 결과로 해당 컬럼을 제외한 나머지 컬럼들의 이름만 들어간 리스트가 하나 새로 만들어지게 된다.
df_use_list = df.columns.difference(['site number','year','PM25'])
df_use_list

Index(['AE', 'AOD', 'CO', 'DAI', 'FMF', 'NDVI', 'NO2', 'O3', 'PBLH', 'PM10',
       'SO2', 'SSA', 'day', 'hour', 'month', 'rh', 'tcc', 'temp', 'wd', 'ws'],
      dtype='object')

In [42]:
df_basic = df[df_useless_list]
df_basic.head()
# 다음과 같이 df뒤 []같은 대괄호 안에 사용할 데이터들의 컬럼 제목들이 들어간 리스트인 df_use_list를 입력해준다
# 다음과 같이 위에서 뺏던 리스트들은 제거된 하나의 데이터 프레임이 만들어지게 된다.

,AE,AOD,CO,DAI,FMF,NDVI,NO2,O3,PBLH,PM10,SO2,SSA,day,hour,month,rh,tcc,temp,wd,ws
0,-999.0,-999.0,0.6,-999.0,-999.0,-999.0,0.023,0.011,994.30627,83.0,0.011,-999.0,1.0,1.0,1.0,65.0,0.77436,-1.2,290.0,3.1
1,-999.0,-999.0,0.6,-999.0,-999.0,-999.0,0.020,0.015,1229.21326,75.0,0.012,-999.0,1.0,2.0,1.0,60.0,0.86685,-1.1,290.0,4.2
2,-999.0,-999.0,0.7,-999.0,-999.0,-999.0,0.022,0.015,1178.61377,86.0,0.012,-999.0,1.0,3.0,1.0,61.0,0.95999,-0.9,270.0,5.6
3,-999.0,-999.0,0.6,-999.0,-999.0,-999.0,0.019,0.019,1161.86670,91.0,0.012,-999.0,1.0,4.0,1.0,49.0,0.80128,-1.1,290.0,5.1
4,-999.0,-999.0,0.4,-999.0,-999.0,-999.0,0.017,0.020,1190.97266,86.0,0.012,-999.0,1.0,5.0,1.0,52.0,0.56124,-2.0,290.0,5.8


In [43]:
# 결과로 나온 이 데이터프레임을 csv로 저장하기 위해서는 df.to_csv() 함수를 사용한다.
# df는 저장할 데이터프레임, ()안에는 경로 및 저장할 파일의 이름과 확장자를 적어주면된다.
# 위의 경우를 예로 들어보면
df_basic.to_csv('/gdrive/My Drive/Colab Notebooks/Model/Dataset/PM10_save/' + 'test_file.csv')
# 해당 경로에 test_file.csv라는 결과 파일이 저장되게 된다.

# 2) 정규화 하기
- 여기서 정규화는 [(원본데이터 - 평균값) / 표준편차] 로 진행하였습니다.

```
# 코드로 형식 지정됨
```



In [44]:
df.head()
# 기본 데이터 값

,site number,year,month,day,hour,AOD,FMF,SSA,AE,NDVI,DAI,SO2,CO,O3,NO2,PM10,PM25,temp,wd,ws,rh,PBLH,tcc
0,111121.0,2012.0,1.0,1.0,1.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.011,0.6,0.011,0.023,83.0,-999.0,-1.2,290.0,3.1,65.0,994.30627,0.77436
1,111121.0,2012.0,1.0,1.0,2.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.012,0.6,0.015,0.020,75.0,-999.0,-1.1,290.0,4.2,60.0,1229.21326,0.86685
2,111121.0,2012.0,1.0,1.0,3.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.012,0.7,0.015,0.022,86.0,-999.0,-0.9,270.0,5.6,61.0,1178.61377,0.95999
3,111121.0,2012.0,1.0,1.0,4.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.012,0.6,0.019,0.019,91.0,-999.0,-1.1,290.0,5.1,49.0,1161.86670,0.80128
4,111121.0,2012.0,1.0,1.0,5.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.012,0.4,0.020,0.017,86.0,-999.0,-2.0,290.0,5.8,52.0,1190.97266,0.56124


In [45]:
df_pm = df['PM10']
df_pm
# Target값이 될 PM10에 대해서는 정규화를 하지 않기 위해 따로 저장해두었습니다

0        83.0
1        75.0
2        86.0
3        91.0
4        86.0
         ... 
22806    21.0
22807    18.0
22808    20.0
22809    23.0
22810    20.0
Name: PM10, Length: 22811, dtype: float64

In [46]:
df_normalize = (df - df.mean()) / df.std()
# 정규화 -> [(원본데이터 - 평균값) / 표준편차]
df_normalize.head()

,site number,year,month,day,hour,AOD,FMF,SSA,AE,NDVI,DAI,SO2,CO,O3,NO2,PM10,PM25,temp,wd,ws,rh,PBLH,tcc
0,NaN,-1.524761,-1.60068,-1.672526,-1.078928,-0.727301,-0.727301,-0.727301,-0.727301,-0.773267,-0.73118,0.172512,0.210531,0.170457,0.206109,0.398488,-1.233833,-0.981192,0.799951,0.036194,0.716773,0.263660,0.604092
1,NaN,-1.524761,-1.60068,-1.672526,-0.638084,-0.727301,-0.727301,-0.727301,-0.727301,-0.773267,-0.73118,0.172517,0.210531,0.170482,0.206094,0.353135,-1.233833,-0.975482,0.799951,0.073247,0.506399,0.800370,0.823499
2,NaN,-1.524761,-1.60068,-1.672526,-0.197240,-0.727301,-0.727301,-0.727301,-0.727301,-0.773267,-0.73118,0.172517,0.211028,0.170482,0.206104,0.415495,-1.233833,-0.964061,0.605432,0.120406,0.548474,0.684762,1.044448
3,NaN,-1.524761,-1.60068,-1.672526,0.243603,-0.727301,-0.727301,-0.727301,-0.727301,-0.773267,-0.73118,0.172517,0.210531,0.170506,0.206089,0.443841,-1.233833,-0.975482,0.799951,0.103563,0.043577,0.646498,0.667953
4,NaN,-1.524761,-1.60068,-1.672526,0.684447,-0.727301,-0.727301,-0.727301,-0.727301,-0.773267,-0.73118,0.172517,0.209537,0.170512,0.206079,0.415495,-1.233833,-1.026874,0.799951,0.127143,0.169801,0.712999,0.098525


In [47]:
df_normalize['PM10'] = df_pm
# pm10값을 기존에 저장해두었던 원본값과 바꿔줍니다.
df_normalize.head()
# 다음과 같이 PM10의 값을 제외하고 전부 표준화가 잘 된것을 확인 할 수 있습니다.

,site number,year,month,day,hour,AOD,FMF,SSA,AE,NDVI,DAI,SO2,CO,O3,NO2,PM10,PM25,temp,wd,ws,rh,PBLH,tcc
0,NaN,-1.524761,-1.60068,-1.672526,-1.078928,-0.727301,-0.727301,-0.727301,-0.727301,-0.773267,-0.73118,0.172512,0.210531,0.170457,0.206109,83.0,-1.233833,-0.981192,0.799951,0.036194,0.716773,0.263660,0.604092
1,NaN,-1.524761,-1.60068,-1.672526,-0.638084,-0.727301,-0.727301,-0.727301,-0.727301,-0.773267,-0.73118,0.172517,0.210531,0.170482,0.206094,75.0,-1.233833,-0.975482,0.799951,0.073247,0.506399,0.800370,0.823499
2,NaN,-1.524761,-1.60068,-1.672526,-0.197240,-0.727301,-0.727301,-0.727301,-0.727301,-0.773267,-0.73118,0.172517,0.211028,0.170482,0.206104,86.0,-1.233833,-0.964061,0.605432,0.120406,0.548474,0.684762,1.044448
3,NaN,-1.524761,-1.60068,-1.672526,0.243603,-0.727301,-0.727301,-0.727301,-0.727301,-0.773267,-0.73118,0.172517,0.210531,0.170506,0.206089,91.0,-1.233833,-0.975482,0.799951,0.103563,0.043577,0.646498,0.667953
4,NaN,-1.524761,-1.60068,-1.672526,0.684447,-0.727301,-0.727301,-0.727301,-0.727301,-0.773267,-0.73118,0.172517,0.209537,0.170512,0.206079,86.0,-1.233833,-1.026874,0.799951,0.127143,0.169801,0.712999,0.098525


# 3) Interpolate 하기

In [48]:
interpolation_columns = df.columns.difference(['site number','year','month','day','hour','PM25'])
# 위에서와 마찬가지로 interpolation하지 않을 데이터들을 빼줍니다.
interpolation_columns

Index(['AE', 'AOD', 'CO', 'DAI', 'FMF', 'NDVI', 'NO2', 'O3', 'PBLH', 'PM10',
       'SO2', 'SSA', 'rh', 'tcc', 'temp', 'wd', 'ws'],
      dtype='object')

In [49]:
df_interpolation = df
# 원본 데이터는 바뀌지 않게 하나의 새로운 데이터 프레임을 만들어줍니다.

In [50]:
df != -999
# null값은 -999로 되어있으므로 다음과 같이 하면 null값이 아닌 데이터들은 'True'로 null값(-999)인 데이터들은 False로 나타나게 됩니다.

,site number,year,month,day,hour,AOD,FMF,SSA,AE,NDVI,DAI,SO2,CO,O3,NO2,PM10,PM25,temp,wd,ws,rh,PBLH,tcc
0,True,True,True,True,True,False,False,False,False,False,False,True,True,True,True,True,False,True,True,True,True,True,True
1,True,True,True,True,True,False,False,False,False,False,False,True,True,True,True,True,False,True,True,True,True,True,True
2,True,True,True,True,True,False,False,False,False,False,False,True,True,True,True,True,False,True,True,True,True,True,True
3,True,True,True,True,True,False,False,False,False,False,False,True,True,True,True,True,False,True,True,True,True,True,True
4,True,True,True,True,True,False,False,False,False,False,False,True,True,True,True,True,False,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22806,True,True,True,True,True,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True
22807,True,True,True,True,True,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True
22808,True,True,True,True,True,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True
22809,True,True,True,True,True,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True


In [52]:
df
# 위의 값과 비교 자료

,site number,year,month,day,hour,AOD,FMF,SSA,AE,NDVI,DAI,SO2,CO,O3,NO2,PM10,PM25,temp,wd,ws,rh,PBLH,tcc
0,111121.0,2012.0,1.0,1.0,1.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.011,0.6,0.011,0.023,83.0,-999.0,-1.2,290.0,3.1,65.0,994.30627,0.77436
1,111121.0,2012.0,1.0,1.0,2.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.012,0.6,0.015,0.020,75.0,-999.0,-1.1,290.0,4.2,60.0,1229.21326,0.86685
2,111121.0,2012.0,1.0,1.0,3.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.012,0.7,0.015,0.022,86.0,-999.0,-0.9,270.0,5.6,61.0,1178.61377,0.95999
3,111121.0,2012.0,1.0,1.0,4.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.012,0.6,0.019,0.019,91.0,-999.0,-1.1,290.0,5.1,49.0,1161.86670,0.80128
4,111121.0,2012.0,1.0,1.0,5.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.012,0.4,0.020,0.017,86.0,-999.0,-2.0,290.0,5.8,52.0,1190.97266,0.56124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22806,111121.0,2019.0,12.0,31.0,1.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.003,0.4,0.024,0.010,21.0,15.0,-8.8,290.0,3.9,34.0,767.63843,0.00000
22807,111121.0,2019.0,12.0,31.0,2.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.002,0.3,0.024,0.011,18.0,13.0,-8.0,320.0,4.3,32.0,808.92407,0.00000
22808,111121.0,2019.0,12.0,31.0,4.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.002,0.3,0.027,0.009,20.0,16.0,-6.1,290.0,2.7,30.0,881.01709,0.00000
22809,111121.0,2019.0,12.0,31.0,5.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.002,0.3,0.026,0.010,23.0,21.0,-5.4,320.0,2.8,30.0,885.85327,0.00000


In [53]:
df[df != -999] 
# 다음과 같이 입력하면 Ture인 값들만 살리고 False인 값들은 모두 제거(NAN으로 바뀜)가 됩니다

,site number,year,month,day,hour,AOD,FMF,SSA,AE,NDVI,DAI,SO2,CO,O3,NO2,PM10,PM25,temp,wd,ws,rh,PBLH,tcc
0,111121.0,2012.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.011,0.6,0.011,0.023,83.0,NaN,-1.2,290.0,3.1,65.0,994.30627,0.77436
1,111121.0,2012.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0.012,0.6,0.015,0.020,75.0,NaN,-1.1,290.0,4.2,60.0,1229.21326,0.86685
2,111121.0,2012.0,1.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,0.012,0.7,0.015,0.022,86.0,NaN,-0.9,270.0,5.6,61.0,1178.61377,0.95999
3,111121.0,2012.0,1.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,0.012,0.6,0.019,0.019,91.0,NaN,-1.1,290.0,5.1,49.0,1161.86670,0.80128
4,111121.0,2012.0,1.0,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,0.012,0.4,0.020,0.017,86.0,NaN,-2.0,290.0,5.8,52.0,1190.97266,0.56124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22806,111121.0,2019.0,12.0,31.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.003,0.4,0.024,0.010,21.0,15.0,-8.8,290.0,3.9,34.0,767.63843,0.00000
22807,111121.0,2019.0,12.0,31.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0.002,0.3,0.024,0.011,18.0,13.0,-8.0,320.0,4.3,32.0,808.92407,0.00000
22808,111121.0,2019.0,12.0,31.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,0.002,0.3,0.027,0.009,20.0,16.0,-6.1,290.0,2.7,30.0,881.01709,0.00000
22809,111121.0,2019.0,12.0,31.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,0.002,0.3,0.026,0.010,23.0,21.0,-5.4,320.0,2.8,30.0,885.85327,0.00000


In [55]:
df[df != -999]['AOD']
# AOD값들의 상당수가 비어있는것을 확인할 수 있습니다.

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
22806   NaN
22807   NaN
22808   NaN
22809   NaN
22810   NaN
Name: AOD, Length: 22811, dtype: float64

In [54]:
df[df != -999]['AOD'].interpolate()
# 다음과 같이 입력하면 초기 값이 없는 곳을 제외하고 모두 linear 하게 interpolate 되게 됩니다.

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
          ...  
22806    0.4552
22807    0.4552
22808    0.4552
22809    0.4552
22810    0.4552
Name: AOD, Length: 22811, dtype: float64

In [59]:
for column in interpolation_columns:
    df_interpolation[column] = df[df != -999][column].interpolate()
df_interpolation
# 다음과 같이 for문을 사용하면 각각의 컬럼들에 대해 interpolation을 진행할 수 있습니다.

,site number,year,month,day,hour,AOD,FMF,SSA,AE,NDVI,DAI,SO2,CO,O3,NO2,PM10,PM25,temp,wd,ws,rh,PBLH,tcc
0,111121.0,2012.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.011,0.6,0.011,0.023,83.0,-999.0,-1.2,290.0,3.1,65.0,994.30627,0.77436
1,111121.0,2012.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0.012,0.6,0.015,0.020,75.0,-999.0,-1.1,290.0,4.2,60.0,1229.21326,0.86685
2,111121.0,2012.0,1.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,0.012,0.7,0.015,0.022,86.0,-999.0,-0.9,270.0,5.6,61.0,1178.61377,0.95999
3,111121.0,2012.0,1.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,0.012,0.6,0.019,0.019,91.0,-999.0,-1.1,290.0,5.1,49.0,1161.86670,0.80128
4,111121.0,2012.0,1.0,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,0.012,0.4,0.020,0.017,86.0,-999.0,-2.0,290.0,5.8,52.0,1190.97266,0.56124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22806,111121.0,2019.0,12.0,31.0,1.0,0.4552,0.4516,0.9657,0.8687,0.1289,0.993,0.003,0.4,0.024,0.010,21.0,15.0,-8.8,290.0,3.9,34.0,767.63843,0.00000
22807,111121.0,2019.0,12.0,31.0,2.0,0.4552,0.4516,0.9657,0.8687,0.1289,0.993,0.002,0.3,0.024,0.011,18.0,13.0,-8.0,320.0,4.3,32.0,808.92407,0.00000
22808,111121.0,2019.0,12.0,31.0,4.0,0.4552,0.4516,0.9657,0.8687,0.1289,0.993,0.002,0.3,0.027,0.009,20.0,16.0,-6.1,290.0,2.7,30.0,881.01709,0.00000
22809,111121.0,2019.0,12.0,31.0,5.0,0.4552,0.4516,0.9657,0.8687,0.1289,0.993,0.002,0.3,0.026,0.010,23.0,21.0,-5.4,320.0,2.8,30.0,885.85327,0.00000


In [62]:
df_interpolation_drop = df_interpolation.dropna()
# dropna 함수를 사용하면 NaN으로 된 값들의 행이 모두 삭제되게됩니다.

In [63]:
df_interpolation_drop.head()

,site number,year,month,day,hour,AOD,FMF,SSA,AE,NDVI,DAI,SO2,CO,O3,NO2,PM10,PM25,temp,wd,ws,rh,PBLH,tcc
17,111121.0,2012.0,1.0,3.0,2.0,0.577600,0.650700,0.962700,1.128800,0.130900,1.243300,0.009,1.2,0.007,0.058,127.0,-999.0,-1.2,70.0,2.3,51.0,595.68970,0.90124
18,111121.0,2012.0,1.0,3.0,3.0,0.565071,0.621671,0.963114,1.090171,0.127957,1.282129,0.011,0.9,0.007,0.053,106.0,-999.0,-0.4,70.0,2.1,45.0,794.23682,0.75064
19,111121.0,2012.0,1.0,3.0,4.0,0.552543,0.592643,0.963529,1.051543,0.125014,1.320957,0.010,0.9,0.006,0.056,95.0,-999.0,-0.6,70.0,1.6,50.0,890.68921,0.87564
20,111121.0,2012.0,1.0,3.0,5.0,0.540014,0.563614,0.963943,1.012914,0.122071,1.359786,0.009,0.7,0.005,0.051,91.0,-999.0,-0.9,270.0,2.5,61.0,1105.08777,0.95624
21,111121.0,2012.0,1.0,3.0,6.0,0.527486,0.534586,0.964357,0.974286,0.119129,1.398614,0.007,0.6,0.005,0.049,82.0,-999.0,-2.0,270.0,3.2,83.0,613.51147,0.97311
